# MODNet: Hole filling method

In this method, we trained the MODNet model on a modified PBE dataset. The values of the modified PBE dataset are predicted using a MODNet model trained on the experimental dataset. This modified PBE dataset contains those predicted values and the experimental data. The experimental data are counted multiple times in order to give more weight to them.

In [1]:
from modnet.preprocessing import MODData
from modnet.models import MODNetModel
import pandas as pd
import numpy as np
import os
import copy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LambdaCallback

In [2]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.loc[ids]
    data.df_targets = data.df_targets.loc[ids]
    data.df_structure = data.df_structure.loc[ids]
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

def MD_append(md,lmd):
    md = copy.deepcopy(md)
    for m in lmd:
        md.df_structure.append(m.df_structure)
        md.df_targets.append(m.df_targets)
        md.df_featurized.append(m.df_featurized)
    return md

In [3]:
md_exp = MODData.load('exp_gap_all')
md_exp.df_targets.columns = ['exp_gap']
md_pbe = MODData.load('pbe_gap_struct.zip')
md_pbe.df_targets.columns = ['pbe_gap']
md_joint = MODData.load('exp_pbe_joint')
#md_hse = MODData.load('moddatas/hse_gap.zip')
#md_hse.df_targets.columns = ['gap']


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f843675b490> object, created with modnet version <=0.1.7
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f840b8c14c0> object, created with modnet version 0.1.8~develop
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f840b8c11f0> object, created with modnet version <=0.1.7


In [4]:
df_exp = md_exp.df_targets

icsds_n =  [int(x.split('-')[1]) for x in list(df_exp.index)]

from pymatgen.ext.matproj import MPRester

with MPRester(api_key='DaZc4G3gfZrVogm6') as mpr:
    res1 = mpr.query({"icsd_ids": {"$in": icsds_n}}, properties=["material_id","icsd_ids"],chunk_size=2000)
    
mapping = {}

for el in res1:
    for icsd in el['icsd_ids']:
        mapping['icsd-'+str(icsd)] = el['material_id']

In [6]:
k = 5
random_state = 202010
folds = MDKsplit(md_exp,n_splits=k,random_state=random_state)
maes_ph1 = np.ones(5)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
        train.df_targets.columns = ['exp_gap']
    else:
        train.feature_selection(n=-1)
        train.save(fpath)
        
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    
    # 1 train on exp only
    
    rlr = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=20, verbose=0, mode="auto", min_delta=0)
    es = EarlyStopping(monitor="loss", min_delta=0.001, patience=300, verbose=0, mode="auto", baseline=None,
                           restore_best_weights=True)
    
    def test_loss(epoch, logs):
        pred = model.predict(test)
        true = test.df_targets
        error = pred-true
        error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
        mae = np.abs(error['exp_gap'].values).mean()
        print('Test loss: {:.3f}'.format(mae))
    test_cb = LambdaCallback(on_epoch_end=test_loss)
        
    
    model = MODNetModel([[['exp_gap']]],{'exp_gap':1}) #,num_neurons=[[350],[64],[8],[8]], n_feat=350,act='elu')
    model.fit_preset(train,verbose=0) # Training the model on the training set
    # model.fit(train, val_fraction = 0.0, lr=0.005, epochs = 1000, batch_size = 64, loss='mae', callbacks=[rlr,es,test_cb], verbose = 0)
    
    pred = model.predict(test)
    true = test.df_targets
    error = pred-true
    error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error['exp_gap'].values).mean()
    print('mae_ph1')
    print(mae)
    maes_ph1[i] = mae
    
    # 2 predict on pbe dataset
    
    df_pred = model.predict(md_pbe) # A lot of materials are in the pbe database. Here we are "filling the holes" by predicting the materials properties of the pbe materials by a model trained on the training set.
    
    df_pred.loc[df_pred['exp_gap']>10,'exp_gap'] = 10 # Unrealistic values above a 10 eV band_gap equal to 10
    df_pred.loc[df_pred['exp_gap']<0,'exp_gap'] = 0 # Unrealistic negative band_gap equal to 0
    print(df_pred['exp_gap'].nlargest(5))
    print(df_pred['exp_gap'].nsmallest(5))
    
    md_pbe_filled = copy.deepcopy(md_pbe) # Copy of the object
    
    md_pbe_filled.df_targets['exp_gap'] = df_pred['exp_gap']
    md_pbe_filled.optimal_features = train.optimal_features
    
    # Replacing the predicted elements in pbe which already exists in the experimental database by their measured experimental properties
    inter_idx = list(set(md_pbe_filled.df_targets.index).intersection(set(train.df_targets.index.map(mapping))))
    train.df_targets = md_joint.df_targets.loc[inter_idx]
    train.df_featurized = md_joint.df_featurized.loc[inter_idx]
    train.df_structure = md_joint.df_structure.loc[inter_idx]
    print(len(train.df_targets))
    
    md_pbe_filled.df_targets.loc[inter_idx[:-200],'exp_gap'] = train.df_targets.loc[inter_idx[:-200],'exp_gap']
    
    ## adding train data with high weight (=copying multiple times)
    print('adding train data with high weight')
    df_temp_targets = pd.DataFrame([])
    df_temp_featurized = pd.DataFrame([])
    df_temp_structure = pd.DataFrame([])
    for j in range(15): # Counting 15 times the experimental data to give more important to these values than the predicted ones
        print(j)
        df_temp_targets = df_temp_targets.append(train.df_targets.iloc[:-200])
        df_temp_featurized = df_temp_featurized.append(train.df_featurized.iloc[:-200])
        df_temp_structure = df_temp_structure.append(train.df_structure.iloc[:-200])
        
    md_pbe_filled.df_targets = md_pbe_filled.df_targets.append(df_temp_targets.append(train.df_targets.iloc[200:]))
    md_pbe_filled.df_featurized = md_pbe_filled.df_featurized.append(df_temp_featurized.append(train.df_featurized.iloc[200:]))
    md_pbe_filled.df_structure = md_pbe_filled.df_structure.append(df_temp_structure.append(train.df_structure.iloc[200:]))
    print('done creating joint df')
    # 3 joint training
    
    rlr = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=20, verbose=1, mode="auto", min_delta=0)
    es = EarlyStopping(monitor="loss", min_delta=0.001, patience=300, verbose=1, mode="auto", baseline=None,
                           restore_best_weights=True)

    model = MODNetModel([[['pbe_gap'],['exp_gap']]],{'exp_gap':0.5, 'pbe_gap':0.5}) #, num_neurons=[[256],[128],[8],[]], n_feat=350,act='elu')
    model.fit_preset(train,verbose=0)
    # model.fit(md_pbe_filled, val_fraction = 200/len(md_pbe_filled.df_targets), val_key='exp_gap', lr=0.005, epochs = 400, batch_size = 64, loss='mae', callbacks=[rlr,es,test_cb], verbose = 1)
    
    pred = model.predict(test)
    true = test.df_targets
    error = pred-true
    error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error['exp_gap'].values).mean()
    print('mae')
    print(mae)
    maes[i] = mae

INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f83f7daa5e0> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.364
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.364
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 

mae_ph1
0.4263014633286549
mp-2064      10.0
mp-17895     10.0
mp-8455      10.0
mp-561543    10.0
mp-4950      10.0
Name: exp_gap, dtype: float32
mp-754225    0.0
mp-21381     0.0
mp-601193    0.0
mp-867164    0.0
mp-982321    0.0
Name: exp_gap, dtype: float32
1358
adding train data with high weight
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
done creating joint df


INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.501
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.359
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.432
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.327
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.413
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.396
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.551
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.472
INFO:root:Training preset #9/16


mae
0.8756824899713055



INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f836fc66880> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.382
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.399
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.347
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.361
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.351
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.385
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.351
INFO:root:Training preset #8/1

...

mae
0.8426753894030112


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f8350cbd220> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.441
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.476
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 2.908
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.754
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.432
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.451
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.379
INFO:root:Training preset #8/1

mae_ph1
0.3551008585966097
mp-1029115    10.0
mp-1028938    10.0
mp-555585     10.0
mp-772008     10.0
mp-1030239    10.0
Name: exp_gap, dtype: float32
mp-581834     0.0
mp-6080       0.0
mp-1029118    0.0
mp-10555      0.0
mp-1029144    0.0
Name: exp_gap, dtype: float32
1374
adding train data with high weight
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
done creating joint df


INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.417
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.399
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.390
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.390
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.391
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.413
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.353
INFO:root:Training preset #8/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.353
INFO:root:Training preset #9/16


mae
0.6334966991072408


In [7]:
train

In [8]:
df_pred['exp_gap']

mp-647999    0.006781
mp-764266    5.039455
mp-19019     0.002770
mp-685999    0.002770
mp-767290    5.161037
               ...   
mp-23292     3.099722
mp-1382      1.912846
mp-628705    0.002770
mp-972072    0.002770
mp-973994    0.002770
Name: exp_gap, Length: 47913, dtype: float32

In [9]:
maes_ph1.mean()

0.39021839978427075

In [10]:
maes.mean()

0.7994939393331272

In [11]:
md_pbe

In [12]:
md_pbe_filled.df_targets

,pbe_gap,exp_gap
mp-647999,1.7050,0.006781
mp-764266,0.8498,5.039455
mp-19019,2.1744,0.002770
mp-685999,0.0000,0.002770
mp-767290,1.8455,5.161037
...,...,...
mp-551,0.0000,0.000000
mp-567928,0.9988,0.430000
mp-554264,2.3711,3.370000
mp-1020029,0.0000,0.000000


#### Conclusion

No improvement.